In [1]:
using Serialization
using Printf
using CSV
using JSON
using DataFrames
using ControlSystemsBase
using ControlTimingSafety
using LinearAlgebra: I
using Distributions: Pareto, Normal, cdf, quantile
using DelimitedFiles

using MATLABControlTest

push!(LOAD_PATH, "../src")
using Experiments
using Benchmarks
using ContinuousSims: nominal_trajectory
using Plots

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [2]:
function get_ref(t::Real, i::Integer)
    t_i = floor(Int64, t / 0.1) + 1
    @boundscheck 1 ≤ t_i ≤ 1000 || throw(ArgumentError("t=$t out of bound"))
    DATA[i]["r"][t_i]
end

function get_period(q::Real, i::Integer)
    e_i = ceil(Int64, q * length(E_VALUES[i]))
    @boundscheck 1 ≤ e_i ≤ length(E_VALUES[i]) || throw(ArgumentError("t=$t out of 		bound"))
    E_VALUES[i][e_i]
end

function get_y(t::Real, i::Integer)
    t_i = floor(Int64, t / 0.1) + 1
    @boundscheck 1 ≤ t_i ≤ 1000 || throw(ArgumentError("t=$t out of bound"))
    DATA[i]["y"][t_i]
end

get_y (generic function with 1 method)

In [3]:
BATCHSIZE = 10
SYS = ss(tf([3, 1],[1, 0.6, 1]))
H = 1000 * 0.1
k = []
Q_VALUES = 0.01:0.01:0.99
file_num = 1:1
DATA = [Dict() for _ in file_num]
E_VALUES =[Float64[] for _ in file_num]
for i in file_num
    file_path = "../O2/output-O2-$i.json"
    DATA[i] = open(file_path) do file
        JSON.parse(file)
    end
    E_VALUES[i] = sort(DATA[i]["t"]) / 100_000 * 0.2
end

In [8]:
for i in file_num
    for q in Q_VALUES
        period = get_period(q, i)
        ref_values_y = map(t -> get_y(t, i), 0:period:H)
        ref_values = map(t -> get_ref(t, i), 0:period:H)
        H_steps = length(ref_values)
        sysd = c2d(SYS, period)
        x0 = zeros(sysd.nx)
        t = generate_samples_mpc_with_multi_ref(sysd, x0, ref_values, ref_values_y, q, BATCHSIZE, H=H_steps)
        a = t[9][3]
        push!(k, a)
    end
end
k

495-element Vector{Any}:
 114566.29071044922
  48274.5458984375
  22553.84423828125
  17334.358032226562
  11691.408752441406
   7652.38623046875
   8058.44677734375
   6224.715728759766
   5312.49755859375
   4363.5341796875
      ⋮
   1223.8368530273438
   1262.91650390625
   1205.8699340820312
   1201.8540649414062
   1197.569091796875
   1262.2941284179688
   1243.7591552734375
   1250.153564453125
   1582.8501586914062